# Проверка кода агента

Используйте ноутбук, чтобы по шагам запускать загрузку моков, выбор фреймворка и кодогенерацию.


In [24]:
from mas_automl.code_agent.load_mocks import load_mock_inputs

DATA_ANALYZE, METADATA, REGISTRY = load_mock_inputs()
print(f"Ключи data: {list[str](DATA_ANALYZE.keys())}")
print(f"Ключи metadata: {list(METADATA.keys())[:5]}")
print(f"Фреймворки: {list(REGISTRY.keys())}")


Ключи data: ['summary', 'priority', 'steps', 'example_pipeline_snippet', 'frameworks_recommended', 'rationale', 'estimated_complexity', 'confidence']
Ключи metadata: ['dataset_id', 'name', 'version', 'version_label', 'description']
Фреймворки: ['AutoGluon', 'H2O AutoML', 'LightAutoML']


In [25]:
METADATA

{'dataset_id': 31,
 'name': 'credit-g',
 'version': 1,
 'version_label': '1',
 'description': '**Author**: Dr. Hans Hofmann  \n**Source**: [UCI](https://archive.ics.uci.edu/ml/datasets/statlog+(german+credit+data)) - 1994    \n**Please cite**: [UCI](https://archive.ics.uci.edu/ml/citation_policy.html)\n\n**German Credit dataset**  \nThis dataset classifies people described by a set of attributes as good or bad credit risks.\n\nThis dataset comes with a cost matrix: \n``` \nGood  Bad (predicted)  \nGood   0    1   (actual)  \nBad    5    0  \n```\n\nIt is worse to class a customer as good when they are bad (5), than it is to class a customer as bad when they are good (1).  \n\n### Attribute description  \n\n1. Status of existing checking account, in Deutsche Mark.  \n2. Duration in months  \n3. Credit history (credits taken, paid back duly, delays, critical accounts)  \n4. Purpose of the credit (car, television,...)  \n5. Credit amount  \n6. Status of savings account/bonds, in Deutsche 

In [19]:
# DATA_ANALYZE, METADATA, REGISTRY
import pandas as pd

PATH_TO_CSV = METADATA.get('local_path')
dataset_df = pd.read_csv(PATH_TO_CSV)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/aleksandr/Documents/projects/MAS_AutoML/experiments/notebooks/data/raw/openml_31_credit-g.csv'

In [3]:
from mas_automl.code_agent.openai_wraper import LLMClient, LLMConfig
from mas_automl.code_agent.base_pipeline import choose_framework

client = LLMClient(LLMConfig())
framework, reason = choose_framework(DATA_ANALYZE, METADATA, REGISTRY, client)
print(framework)
print(reason)


AutoGluon
Эвристика на основе типа задачи и размера выборки.


In [4]:
from mas_automl.code_agent.base_pipeline import generate_code, evaluate_code

code_sample = generate_code(framework, client, iteration=1, feedback="")
print(code_sample)
print(evaluate_code(code_sample, framework))


import pandas as pd
from autogluon.tabular import TabularPredictor

def train_model(train_df: pd.DataFrame, test_df: pd.DataFrame, label: str) -> TabularPredictor:
    predictor = TabularPredictor(label=label)
    predictor.fit(train_data=train_df, presets='medium_quality', time_limit=60)
    predictor.evaluate(test_df, silent=True)
    return predictor
(True, 'Проверки пройдены.')


In [5]:
from mas_automl.code_agent.base_pipeline import run_pipeline

result = run_pipeline(max_iterations=2, llm=client)
result


PipelineResult(framework='AutoGluon', reason='Эвристика на основе типа задачи и размера выборки.', code="import pandas as pd\nfrom autogluon.tabular import TabularPredictor\n\ndef train_model(train_df: pd.DataFrame, test_df: pd.DataFrame, label: str) -> TabularPredictor:\n    predictor = TabularPredictor(label=label)\n    predictor.fit(train_data=train_df, presets='medium_quality', time_limit=60)\n    predictor.evaluate(test_df, silent=True)\n    return predictor", tests_passed=True, iterations=1, feedback='Проверки пройдены.')